코드를 혼자서는 짜기 막막하네요... 많이 참고했고 별로 진도를 못 나갔습니다.

# Goals

- 단일경구약제 중 TL_81 파일의 것만 학습/분류 (13GB)
- 추후 무슨 종류의 약을 어떤 이유로 학습시키고 싶은지 구체적으로 정하도록

# Done

* 2023/01/11
- import json file and train dataset image names
- Resize train dataset images to 64 x 64

# To-do

* TBD:
- Process data from json and change to csv (np array?)
> Tags: file_name, dl+name, dl_material
- Set Model (Keras)
> input_shape: (64,64,3), softmax(classification)
- Train Data
> Visualize results of Loss/Accuracy w/ matplotlib
- Compare prediction accuracy before/after training

In [12]:
import cv2, json, glob, os, time
import numpy as np
from matplotlib import pyplot as plt
cv2.__version__

'4.7.0'

In [12]:
#IMG filepath save - DEPRECATED

img_path=('./1.Training/images/')
img_files = list(os.listdir(img_path))
print("Number of image files: {}".format(len(img_files)))

img_filepath=[]

for each in img_files:
    img_indiv=(f"{img_path}./1.Training/원천데이터/단일경구약제 5000종/{str(each)}")
    img_filepath.append(img_indiv)
print(img_filepath[:5])

Number of image files: 24
['./1.Training/images/./1.Training/원천데이터/단일경구약제 5000종/K-038884', './1.Training/images/./1.Training/원천데이터/단일경구약제 5000종/K-038890', './1.Training/images/./1.Training/원천데이터/단일경구약제 5000종/K-038896', './1.Training/images/./1.Training/원천데이터/단일경구약제 5000종/K-038910', './1.Training/images/./1.Training/원천데이터/단일경구약제 5000종/K-038912']


In [4]:
#JSON filepath save - DEPRECATED

json_path=('./1.Training/라벨링데이터/단일경구약제 5000종/K-038884_json')
json_files = list(os.listdir(json_path))
print("Number of json files: {}".format(len(json_files)))

file_name=[]

for i in range(len(json_files)):
    json_each=json_path+'/'+json_files[i]
    # print(json_files[i])
    with open(json_each, 'r', encoding='utf-8') as j:
        js=json.load(j)
        file_name=js['images'][0]['file_name']
        med_name=js['images'][0]['dl_name']
        med_type=js['images'][0]['dl_material']
        print(file_name, med_name, med_type, sep=' // ', end='\n')

Number of json files: 216
K-038884_0_0_0_0_75_000_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_020_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_040_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_060_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_080_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_100_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_120_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_140_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_160_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_180_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_200_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_220_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_240_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_260_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_280_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_300_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_320_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_75_340_200.png // 에피나레정 // 에피나스틴염산염
K-038884_0_0_0_0_90_

In [5]:
#ALTERNATIVE

dir_1 = glob.glob('./1.Training/라벨링데이터/단일경구약제 5000종/*')
json_names = []
for temp_dir in dir_1:
    json_names += glob.glob(temp_dir + '/*.*')
print(json_names[:5])

temp_img_dir = glob.glob('./1.Training/images/*')
# temp_img_dir = glob.glob('./1.Training/원천데이터/단일경구약제 5000종/*')
# print(len(temp_img_dir))  #should be 24
img_names = []
for temp in temp_img_dir:
    img_names += glob.glob(temp + '/*.*')
print(img_names[:5])

['./1.Training/라벨링데이터/단일경구약제 5000종\\K-038884_json\\K-038884_0_0_0_0_75_000_200.json', './1.Training/라벨링데이터/단일경구약제 5000종\\K-038884_json\\K-038884_0_0_0_0_75_020_200.json', './1.Training/라벨링데이터/단일경구약제 5000종\\K-038884_json\\K-038884_0_0_0_0_75_040_200.json', './1.Training/라벨링데이터/단일경구약제 5000종\\K-038884_json\\K-038884_0_0_0_0_75_060_200.json', './1.Training/라벨링데이터/단일경구약제 5000종\\K-038884_json\\K-038884_0_0_0_0_75_080_200.json']
['./1.Training/images\\K-038884\\K-038884_0_0_0_0_75_000_200.png', './1.Training/images\\K-038884\\K-038884_0_0_0_0_75_020_200.png', './1.Training/images\\K-038884\\K-038884_0_0_0_0_75_040_200.png', './1.Training/images\\K-038884\\K-038884_0_0_0_0_75_060_200.png', './1.Training/images\\K-038884\\K-038884_0_0_0_0_75_080_200.png']


In [14]:
#IMG_RESIZE
#조장 김지수님 코드 기반

i = 0
# os.mkdir('Data/1.Training/images/' + 'hi')
for idx, img_data in enumerate(json_names):
    img_data = img_data.replace('\\', '/', 10)
    # print(img_data)
    with open(img_data, 'r', encoding='UTF=8') as j:
        js = json.load(j)
        file_name = js['images'][0]['file_name']
        file_dir = './1.Training/images/' + file_name[:8]
        # print(file_dir)
        if not os.path.isdir(file_dir):
            print('Saving directory:',file_dir) #추가
            os.mkdir(file_dir)

        file_dir = file_dir + '/' + file_nameW

        if not os.path.isfile(file_dir):
            # print(file_dir)
            x, y, w, h = js['annotations'][0]['bbox']

            cx = x + w // 2
            cy = y + h // 2
            w_h, h_h = max(w, h), max(w, h)  # 깨끗한 data 를 얻기 위해 크롭영역을 정방형으로 해준다. 기준은 둘 중 큰 값
            x = cx - (w_h // 2)
            y = cy - (h_h // 2)

            # print(x, y, w, h)
            img_names[idx] = img_names[idx].replace('\\', '/', 10)
            # print(img_names[idx])
            img = cv2.imread(img_names[idx], cv2.IMREAD_COLOR)
            
            cropped_img = img[y:y + h_h, x:x + w_h]
            cropped_img = cv2.resize(cropped_img, (64, 64))
            if idx % 100 == 0:
                print('진행상황: {} / {}'.format(idx, len(img_names)))
            # cv2.imwrite(file_dir, cropped_img)
        else:
            print(f'Data named {file_dir} already processed.')
            continue

Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_000_200.png already processed.
Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_020_200.png already processed.
Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_040_200.png already processed.
Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_060_200.png already processed.
Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_080_200.png already processed.
Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_100_200.png already processed.
Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_120_200.png already processed.
Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_140_200.png already processed.
Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_160_200.png already processed.
Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_180_200.png already processed.
Data named ./1.Training/images/K-038884/K-038884_0_0_0_0_75_200_200.png already processed.